# Load libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import sys
import argparse
import csv
from scipy.sparse import *
import pandas as pd
import regex 
from sklearn.base import TransformerMixin
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.util import ngrams
from sklearn.model_selection import StratifiedKFold, train_test_split


In [ ]:
from  sklearn.metrics import f1_score,accuracy_score,precision_score,classification_report

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation,GRU
from keras.layers import Bidirectional, GlobalMaxPool1D,Concatenate,Conv1D,SpatialDropout1D,BatchNormalization,concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import re

In [ ]:
!pip install emoji

     |████████████████████████████████| 51kB 3.6MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49717 sha256=30c4444227e9b9fbe61cbbbb1586f6ec10ff713181023e8969d57ad3437d5753
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [ ]:
import emoji


In [ ]:
from collections import Counter, OrderedDict


In [ ]:
import math

In [ ]:
import random
from random import seed

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.optimizers import Adam


In [ ]:
from keras import backend as K


In [ ]:
import tensorflow as tf

In [ ]:
from keras.layers import Activation, Permute, RepeatVector, Lambda
from keras.layers import Dense, Dropout, Flatten, LSTM, GRU, Bidirectional, Input, Multiply
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras import regularizers



In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


# Helper Functions

In [ ]:
from gensim.models import KeyedVectors
def load_w2v(filepath,binary):
    return KeyedVectors.load_word2vec_format(filepath, binary=binary)

In [ ]:
def clean_text(text):  
    # ref: https://github.com/bakrianoo/aravec
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
    
    tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(tashkeel,"", text)
    
    # longation = re.compile(r'(.)\1+')
    # subst = r"\1\1"
    # text = re.sub(longation, subst, text)
    
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"[a-zA-Z]", '', text)
    text = re.sub(r"\d+", ' ', text)
    text = re.sub(r"\n+", ' ', text)
    text = re.sub(r"\t+", ' ', text)
    text = re.sub(r"\r+", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    text = text.strip()
    
    return text

In [ ]:
# Get the vec representation of a set of tweets based on a specified embedding (can be a word or emoji mapping)
def get_tweets_embeddings(tweets, vec_map, embedding_dim=100, init_unk=False, variance=None, weighted_average=True):
    # Get the variance of the embedding map
    if init_unk and variance is None:
        variance = embedding_variance(vec_map)
        print("Vector mappings have variance ", variance)
    # If set, calculate the tf-idf weight of each embedding, otherwise, no weighting (all weights are 1.0)
    if weighted_average:
        weights = get_tf_idf_weights(tweets, vec_map)
    else:
        weights = {k: 1.0 for k in vec_map.vocab}
    tw_emb = np.zeros((len(tweets), embedding_dim))
    for i, tw in enumerate(tweets):
        total_valid = 0
        for word in tw.split():
            embedding_vector=None 
            if word in vec_map.wv:
              embedding_vector = vec_map[word]
            if embedding_vector is not None:
                tw_emb[i] = tw_emb[i] + embedding_vector * weights[word]
                total_valid += 1
            elif init_unk:
                seed(1337603)
                tw_emb[i] = np.random.uniform(-variance, variance, size=(1, embedding_dim))
            # else:
            #    print("Not found: ", word)
        # Get the average embedding representation for this tweet
        tw_emb[i] /= float(max(total_valid, 1))
    return tw_emb


In [ ]:
# Get the tf-idf weighting scheme (used to measure the contribution of a word in a tweet => weighted sum of embeddings)
def get_tf_idf_weights(tweets, vec_map):
    df = {}
    for tw in tweets:
        words = set(tw.split())
        for word in words:
            if word not in df:
                df[word] = 0.0
            df[word] += 1.0
    idf = OrderedDict()
    for word in vec_map.vocab:
        n = 1.0
        if word in df:
            n += df[word]
        score = math.log(len(tweets) / float(n))
        idf[word] = score
    return idf


# Compute the similarity of 2 vectors, both of shape (n, )
def cosine_similarity(u, v):
    dot = np.dot(u, v)
    norm_u = np.sqrt(np.sum(u ** 2))
    norm_v = np.sqrt(np.sum(v ** 2))
    cosine_distance = dot / (norm_u * norm_v)
    return cosine_distance


# Convert emojis to unicode representations by removing any variation selectors
# Info: http://www.unicode.org/charts/PDF/UFE00.pdf
def convert_emoji_to_unicode(emoji):
    unicode_emoji = emoji.encode('unicode-escape')
    find1 = unicode_emoji.find(b"\\ufe0f")
    unicode_emoji = unicode_emoji[:find1] if find1 != -1 else unicode_emoji
    find2 = unicode_emoji.find(b"\\ufe0e")
    unicode_emoji = unicode_emoji[:find2] if find2 != -1 else unicode_emoji
    return unicode_emoji


# Performs the word analogy task: a is to b as c is to ____.
def make_analogy(a, b, c, vec_map):
    a = convert_emoji_to_unicode(a)
    b = convert_emoji_to_unicode(b)
    c = convert_emoji_to_unicode(c)

    e_a, e_b, e_c = vec_map[a], vec_map[b], vec_map[c]

    max_cosine_sim = -100
    best = None
    best_list = {}
    for v in vec_map.keys():
        # The best match shouldn't be one of the inputs, so pass on them.
        if v in [a, b, c]:
            continue
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)
        cosine_sim = cosine_similarity(e_b - e_a, vec_map[v] - e_c)
        best_list[v] = cosine_sim
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best = v
    sorted_keys = sorted(best_list, key=best_list.get, reverse=True)
    print("Top 5 most similar emojis: ", [r.decode('unicode-escape') for r in sorted_keys[:5]])
    print(str.format('{} - {} + {} = {}', a.decode('unicode-escape'), b.decode('unicode-escape'),
                     c.decode('unicode-escape'), best.decode('unicode-escape')), "\n\n")


# Get the Euclidean distance between two vectors
def euclidean_distance(u_vector, v_vector):
    distance = np.sqrt(np.sum([(u - v) ** 2 for u, v in zip(u_vector, v_vector)]))
    return distance


# Given a tweet, return the scores of the most similar/dissimilar pairs of words
def get_similarity_measures(tweet, vec_map, weighted=False, verbose=True):
    # Filter a bit the tweet so that no punctuation and no stopwords are included
    stopwords = data_proc.get_stopwords_list()
    filtered_tweet = list(set([w.lower() for w in tweet.split()
                               if w.isalnum() and w not in stopwords and w.lower() in vec_map.keys()]))
    # Compute similarity scores between any 2 words in filtered tweet
    similarity_scores = []
    max_words = []
    min_words = []
    max_score = -100
    min_score = 100
    for i in range(len(filtered_tweet) - 1):
        wi = filtered_tweet[i]
        for j in range(i + 1, len(filtered_tweet)):
            wj = filtered_tweet[j]
            similarity = cosine_similarity(vec_map[wi], vec_map[wj])
            if weighted:
                similarity /= euclidean_distance(vec_map[wi], vec_map[wj])
            similarity_scores.append(similarity)
            if max_score < similarity:
                max_score = similarity
                max_words = [wi, wj]
            if min_score > similarity:
                min_score = similarity
                min_words = [wi, wj]
    if verbose:
        print("Filtered tweet: ", filtered_tweet)
        if max_score != -100:
            print("Maximum similarity is ", max_score, " between words ", max_words)
        else:
            print("No max! Scores are: ", similarity_scores)
        if min_score != 100:
            print("Minimum similarity is ", min_score, " between words ", min_words)
        else:
            print("No min! Scores are: ", similarity_scores)
    return max_score, min_score


# Custom metric function adjusted from https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def f1_score(y_true, y_pred):
    # Recall metric. Only computes a batch-wise average of recall,
    # a metric for multi-label classification of how many relevant items are selected.
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    # Precision metric. Only computes a batch-wise average of precision,
    # a metric for multi-label classification of how many selected items are relevant.
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision*recall) / (precision+recall))


# This code allows you to see the mislabelled examples
def analyse_mislabelled_examples(x_test, y_test, y_pred):
    for i in range(len(y_test)):
        num = np.argmax(y_pred[i])
        if num != y_test[i]:
            print('Expected:', y_test[i], ' but predicted ', num)
            print(x_test[i])
# Extract each tweet's emojis - obv. it's just a brute force solution (so, it's slow) but works in ALL cases
def extract_emojis(tweets):
    emojis = []
    for tw in tweets:
        tw_emojis = []
        for word in tw:
            chars = list(word)
            for ch in chars:
                if ch in emoji.UNICODE_EMOJI:
                    tw_emojis.append(ch)
        emojis.append(' '.join(tw_emojis))
    return emojis

In [ ]:
# Calculate the variance of an embedding (like glove, word2vec, emoji2vec, etc)
# Used to sample new uniform distributions of vectors in the interval [-variance, variance]
def embedding_variance(vec_map):
    variance = np.sum([np.var(vec) for vec in vec_map.vectors]) / len(vec_map.vectors)
    return variance


In [ ]:
# Get some idea about the max and mean length of the tweets (useful for deciding on the sequence length)
def get_max_len_info(tweets, average=False):
    sum_of_length = sum([len(l.split()) for l in tweets])
    avg_tweet_len = sum_of_length / float(len(tweets))
    print("Mean of train tweets: ", avg_tweet_len)
    max_tweet_len = len(max(tweets, key=len).split())
    print("Max tweet length is = ", max_tweet_len)
    if average:
        return avg_tweet_len
    return max_tweet_len

In [ ]:
def encode_text_as_word_indexes(train_tweets, test_tweets , max_num_words=None, lower=False, char_level=False):
    # Create the tokenizer
    tokenizer = Tokenizer(num_words=max_num_words, filters='', lower=lower, split=" ", char_level=char_level)
    # Fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_tweets)
    # Encode each example as a sequence of word indexes based on the vocabulary of the tokenizer
    x_train = tokenizer.texts_to_sequences(train_tweets)
    x_test=tokenizer.texts_to_sequences(test_tweets)
   
    return tokenizer, x_train,x_test

In [ ]:
def convert_emojis(text):
    emoji___=''.join(c for c in text if c in emoji.UNICODE_EMOJI)
    return emoji.demojize(emoji___)

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [ ]:
def contain_emoji(tweets):

  emoji___=''.join(c for c in tweets if c in emoji.UNICODE_EMOJI)
  # print(emoji___)
  # print(tweets)
  if len(emoji___)>0:
    return True
  return False

In [ ]:
def transform(df):
    
    df["word_count"] = df["cleaned_text"].apply(lambda x: len(x.split()))
    
    df["contain_emoji"] = False
    
    df["demoji_text"] = ""
    for index, row in df.iterrows():
      # df.loc[index, 'cleaned_text']=remove_urls(row["cleaned_text"])
      if contain_emoji(row["tweet"]):
        df.loc[index, 'contain_emoji']=True
        df.loc[index, 'demoji_text']=convert_emojis(row["tweet"])
        # print(convert_emojis(row["cleaned_text"]))


    
    return df

In [ ]:
# Compute the word-embedding matrix
def get_embedding_matrix(word2vec_map, word_to_index, embedding_dim, init_unk=True, variance=None):
    # Get the variance of the embedding map
    if init_unk and variance is None:
        variance = embedding_variance(word2vec_map)
        print("Word vectors have variance ", variance)
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors)
    embedding_matrix = np.zeros((len(word_to_index) + 1, embedding_dim))
    for word, i in word_to_index.items():
        embedding_vector=None 
        if word in word2vec_map.wv:
          embedding_vector = word2vec_map[word]

        # embedding_vector = word2vec_map.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        elif init_unk:
            # Unknown tokens are initialized randomly by sampling from a uniform distribution [-var, var]
            seed(1337603)
            embedding_matrix[i] = np.random.uniform(-variance, variance, size=(1, embedding_dim))
        # else:
        #    print("Not found: ", word)
    return embedding_matrix

In [ ]:
def keras_dl( word2vec_embegging,emoji_embedd,voacb_shape_embdd,input_shape_1 ):   
    inp = Input(shape = (input_shape_1,), name = 'text_clean')
    x = Embedding(voacb_shape_embdd,300,weights = [word2vec_embegging], trainable = False)(inp)
    # x1 = SpatialDropout1D(0.5)(x)
    
    x_lstm = Bidirectional(LSTM(128, return_sequences = True))(x)
    x_lstm_c1d = Conv1D(64,kernel_size=3,padding='valid',activation='tanh')(x_lstm)
    x_lstm_c1d_gp = GlobalMaxPooling1D()(x_lstm_c1d)
    
    x_gru = Bidirectional(GRU(128, return_sequences = True))(x)
    x_gru_c1d = Conv1D(64,kernel_size=2,padding='valid',activation='tanh')(x_gru)
    x_gru_c1d_gp = GlobalMaxPooling1D()(x_gru_c1d)

    inp2 = Input(shape = (input_shape_1,), name = 'emoji')
    x2 = Embedding(voacb_shape_embdd,300,weights = [emoji_embedd], trainable = False)(inp2)
    X2_conv1 = Conv1D(64, 5, kernel_initializer='he_normal', padding='valid', activation='relu')(x2)
    X2_lstm = LSTM(16, kernel_initializer='he_normal', activation='tanh',
             dropout=0.5, return_sequences=True)(X2_conv1)
    x2_flatten = Flatten()(X2_lstm)
    x2_dense =(Dense(32, activation='tanh') (x2_flatten))  
    

    
    x_f = concatenate([x_lstm_c1d_gp, x_gru_c1d_gp])
    x_f = BatchNormalization()(x_f)
    x_f =(Dense(128, activation='tanh') (x_f))    
    x_f = BatchNormalization()(x_f)
    x_f = concatenate([x_f, x2_dense])    
    x_f = (Dense(64, activation='tanh') (x_f))
    x_f = Dense(2, activation = "sigmoid")(x_f)
    model = Model(inputs = [inp, inp2], outputs = x_f)
    return model


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")

Mounted at /content/drive


In [ ]:
train_data=pd.read_csv('/content/drive/MyDrive/Sentiment_Analysis_Arabic/Dataset/training_data.csv')

In [ ]:
test_data=pd.read_csv('/content/drive/MyDrive/Sentiment_Analysis_Arabic/Dataset/ArSarcasm_test.csv')


In [ ]:
test_data.head()

,dialect,sarcasm,sentiment,original_sentiment,tweet,source
0,msa,True,negative,negative,"""@AbuEmad74241481 @Cesars2014 هههههه حزب الله ...",semeval
1,gulf,False,positive,neutral,"""RT @JannetForster: البنات اللي م صامو بقولكم ...",semeval
2,msa,True,neutral,neutral,اشارة رابعة اشبه بنار تحرق الانقلابيين,astd
3,msa,False,neutral,neutral,"""@EGYPTAIR ماهي مميزات درجه بزنس علماً اني في ...",semeval
4,msa,False,neutral,neutral,ما لا تراه على التلفاز منافسة شديدة بين المرشح...,semeval


In [ ]:
train_data.head()

,tweet,sarcasm,sentiment,dialect
0,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",False,NEU,msa
1,"""مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...",False,NEU,msa
2,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,True,NEG,msa
3,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",True,NEG,gulf
4,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",False,NEU,msa


# Prepare Data

In [ ]:
train_data['cleaned_text'] = train_data.tweet.apply(clean_text)
test_data['cleaned_text'] = test_data.tweet.apply(clean_text)
train_data['cleaned_text'] = train_data.cleaned_text.apply(remove_urls)
test_data['cleaned_text'] = test_data.cleaned_text.apply(remove_urls)


In [ ]:
train_data = transform(train_data)
test_data = transform(test_data)

In [ ]:
train_data

,tweet,sarcasm,sentiment,dialect,cleaned_text,word_count,contain_emoji,demoji_text
0,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",False,NEU,msa,د محمود العلايلياري ان الفريق احمد شفيق رقم مه...,22,False,
1,"""مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...",False,NEU,msa,مع فيدرر يااجا والكبار,4,True,:smiling_face_with_heart-eyes:
2,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,True,NEG,msa,الداعون لمبدا الاختلاط بين الجنسين كالداعين لا...,16,False,
3,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",True,NEG,gulf,مساكين من الصبح وهوما رايحين راجعين عاي غوغل ت...,12,False,
4,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",False,NEU,msa,قل شرق حلب ولا تقل حلب الشرقيه وقل غرب حلب ولا...,20,False,
...,...,...,...,...,...,...,...,...
12543,هلأ صاير انت يا #فلعوط بدك تعطي محاضرات ع تويت...,True,NEG,levant,هلا صاير انت يافلعوط بدك تعطي محاضرات ع تويتر ...,28,False,
12544,"""لا اله الا الله💜#أيفون_البروفيسور https://t.c...",False,NEU,egypt,لا اله الا اللهايفون البروفيسور,5,True,:purple_heart:
12545,"""RT @turkyepost: #أردوغان : إذا كان المرتكب غي...",False,NEU,msa,اردوغان اذا كان المرتكب غير مسلم لا يسمونها عم...,19,False,
12546,RT @Yousiif65: هاري بوتر؟👓🎩 https://t.co/959Oo...,False,POS,egypt,هاري بوتر,2,True,:glasses::top_hat:


In [ ]:
# Transform the output into categorical data or just keep it as it is (in a numpy array)
train_labels_task1 = to_categorical(np.asarray(train_data['sarcasm']))
# train_labels_task2 = to_categorical(np.asarray(train_data['sentiment'].factorize()[0]))




In [ ]:
# Transform the output into categorical data or just keep it as it is (in a numpy array)
test_labels_task1 = to_categorical(np.asarray(test_data['sarcasm']))
# test_labels_task2 = to_categorical(np.asarray(test_data['sentiment'].factorize()[0]))




### Load Embeddings

In [ ]:
MAX_SEQUENCE_LENGTH = 300 #176 # based on our inputs; TODO: remove outliers? dynamically calculate!
MAX_NB_WORDS = len(word2vec_map.vocab)
EMBEDDING_DIM = 300 # w2v, fastText; GloVe=50

In [ ]:
word2vec_map = load_w2v("/content/drive/MyDrive/Sentiment_Analysis_Arabic/Dataset/arabic-news.bin", binary=True)

In [ ]:
emoji2vec_map = load_w2v("/content/drive/MyDrive/Sentiment_Analysis_Arabic/Dataset/emoji2vec.bin",True)


#### Transform Data  vector2index

In [ ]:
# Get the max length of the train tweets
max_tweet_length = train_data.cleaned_text.map(len).max()

# Convert all tweets into sequences of word indices
tokenizer, train_indices,test_indices = encode_text_as_word_indexes(train_data['cleaned_text'],test_data['cleaned_text'],  lower=True)
vocab_size = len(tokenizer.word_counts) + 1
word_to_index = tokenizer.word_index
print("There are %s unique tokens." % len(word_to_index))

# Pad sequences with 0s (can do it post or pre - post works better here)
x_train = pad_sequences(train_indices, maxlen=max_tweet_length, padding="post", truncating="post", value=0.)
x_test = pad_sequences(test_indices, maxlen=max_tweet_length, padding="post", truncating="post", value=0.)



# Get the word to index and the index to word mappings
word_index = tokenizer.word_index
index_to_word = {index: word for word, index in word_index.items()}

# Set the sequence length
SEQUENCE_LENGTH = max_tweet_length


There are 45466 unique tokens.


In [ ]:
# Convert all tweets into sequences of word indices
tokenizer_emji, train_indices_emji,test_indices_emji = encode_text_as_word_indexes(train_data['demoji_text'],test_data['demoji_text'],  lower=True)
word_to_index_emji= tokenizer_emji.word_index
print("There are %s unique tokens." % len(word_to_index_emji))

# Pad sequences with 0s (can do it post or pre - post works better here)
x_train_emji = pad_sequences(train_indices_emji, maxlen=max_tweet_length, padding="post", truncating="post", value=0.)
x_test_emji = pad_sequences(test_indices_emji, maxlen=max_tweet_length, padding="post", truncating="post", value=0.)



# Get the word to index and the index to word mappings
word_index_emji = tokenizer_emji.word_index
index_to_word_emji = {index: word for word, index in word_index_emji.items()}



There are 799 unique tokens.


# Prepare Model

In [ ]:
w2v_embedd=get_embedding_matrix(word2vec_map,word_index,300)

Word vectors have variance  0.7012621721375845


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
emoji_embedd=get_embedding_matrix(emoji2vec_map,word_index,300)

Word vectors have variance  0.0033188830794231637


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
model=keras_dl(w2v_embedd,emoji_embedd,w2v_embedd.shape[0],max_tweet_length)

In [ ]:
my_optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.99, decay=0.01)
reduceLR = ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=3, verbose=1)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=my_optimizer, metrics=['accuracy',f1_m,precision_m, recall_m])

#### Train Model

In [ ]:
# Fit the model on the training data
history = model.fit([x_train,x_train_emji], train_labels_task1, batch_size=32, epochs=100, shuffle=True,
                             validation_split=0.1, verbose=1)

Epoch 1/10
353/353 [==============================] - 40s 88ms/step - loss: 0.5837 - accuracy: 0.7112 - f1_m: 0.6771 - precision_m: 0.6795 - recall_m: 0.6766 - val_loss: 0.4450 - val_accuracy: 0.8112 - val_f1_m: 0.7780 - val_precision_m: 0.8002 - val_recall_m: 0.7596
Epoch 2/10
353/353 [==============================] - 29s 81ms/step - loss: 0.3686 - accuracy: 0.8437 - f1_m: 0.8096 - precision_m: 0.8148 - recall_m: 0.8059 - val_loss: 0.4424 - val_accuracy: 0.8096 - val_f1_m: 0.7695 - val_precision_m: 0.7868 - val_recall_m: 0.7565
Epoch 3/10
353/353 [==============================] - 29s 82ms/step - loss: 0.3412 - accuracy: 0.8583 - f1_m: 0.8240 - precision_m: 0.8283 - recall_m: 0.8212 - val_loss: 0.4407 - val_accuracy: 0.8112 - val_f1_m: 0.7747 - val_precision_m: 0.7916 - val_recall_m: 0.7619
Epoch 4/10
353/353 [==============================] - 28s 81ms/step - loss: 0.3258 - accuracy: 0.8640 - f1_m: 0.8316 - precision_m: 0.8350 - recall_m: 0.8298 - val_loss: 0.4399 - val_accuracy: 0.8

### Test Model

In [ ]:
model.evaluate([x_test,x_test_emji],test_labels_task1)

66/66 [==============================] - 2s 33ms/step - loss: 0.1486 - accuracy: 0.9578 - f1_m: 0.9348 - precision_m: 0.9352 - recall_m: 0.9356


[0.14855192601680756,
 0.9578198790550232,
 0.9348338842391968,
 0.9352262020111084,
 0.9355745315551758]

In [ ]:
predictions=model.predict([x_test,x_test_emji])

In [ ]:
argsorted_y = np.argsort(predictions)[:,-1:]

In [ ]:
print(classification_report(test_labels_task1.argmax(axis=1), argsorted_y))